In [1]:
# Algumas Definições e Funções de Suporte
import random, math, fractions, sys, cProfile, pstats, os
from IPython.display import display, Markdown

def print_primes(p, q):
    display(Markdown("""
# Números primos gerados
| variável | número primo |
|---|-----|
| p | {p} |
| q | {q} |
    """.format(p = p, q = q)))

def print_keys(e, n, d):
    display(Markdown("""
# Chaves geradas
--------------------------------
| Tipo de Chave | Chave Gerada |
|---------------|--------------|
| Pública       | {e}          |
| Compartilhada | {n}          |
| Privada       | {d}          |
--------------------------------
    """.format(e=e, n=n, d=d)))
    
def print_euctab(tab):
    str = """
# Passos do Algoritmo Extendido de Euclides
---------------------------------------------------------------------
| Dividendo | Divisor | Quociente | Resto Anterior | Novo Resto | t |
|:---------:|:-------:|:---------:|:--------------:|:----------:|:-:|
    """

    for i in range(len(tab)):
        str += "|{a}|{b}|{c}|{d}|{e}|{f}|".format(a = tab[i][0], b = tab[i][1], c = tab[i][2], d = tab[i][3], e = tab[i][4], f = tab[i][5]) + os.linesep
    str += """
-----------------------------------------------------------------
    """
    display(Markdown(str))

def print_sec(enc, dec):
    str = """
# Resultados da Encriptação e Decriptação
---------------------------
| Encriptado | Decriptado |
|------------|------------|
    """
    
    for i in range(len(enc)):
        str += "| {a} | {b} |".format(a = enc[i], b = dec[i]) + os.linesep
    str += """
---------------------------    
    """
    display(Markdown(str))     

In [2]:
# Peneira de Eratóstenes:
# https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes
def generate_small_primes(max_prime):
    sieve = [True for i in range(0, max_prime)]
    for i in range(2, int(math.sqrt(max_prime))):
        if sieve[i]:
            for j in range(i**2, max_prime, i):
                if (j < max_prime):
                    sieve[j] = False

    return [i for i in range(2, max_prime) if sieve[i]]

prime_cache = generate_small_primes(100000)
display(Markdown("# Primeiros 19 pequenos primos gerados:"))
display([prime_cache[i] for i in range(19)])
display(Markdown("# Ultimos 11 pequenos primos gerados"))
display([prime_cache[i] for i in range(len(prime_cache) - 11, len(prime_cache))])

# Primeiros 19 pequenos primos gerados:

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67]

# Ultimos 11 pequenos primos gerados

[99871, 99877, 99881, 99901, 99907, 99923, 99929, 99961, 99971, 99989, 99991]

In [3]:
# Teste de Primalidade de Fermat:
# https://en.wikipedia.org/wiki/Fermat_primality_test
def is_prime_fermat(number):
    for i in range(0, 3):
        a = random.randint(2, number-2)
        if pow(a, number-1, number) != 1:
            return False
    return True

In [4]:
# Teste de forte pseudo-primalidade de Miller-Rabin
# https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test
# Handbook of Applied Cryptography, seção 4.2.3 (http://cacr.uwaterloo.ca/hac/)
def is_prime_millerrabin(number, tries):    
    # Encontra n-1 = d = 2**s * r tal que r é impar
    s = 0
    d = number - 1
    while d % 2 == 0:
        d >>= 1
        s += 1

    # Realiza 'tries' tentativas, testando as testemunhas de
    # multiplicidade do numero
    for i in range(tries):
        a = random.randrange(2, number - 1)
        if not check_witness(a, s, d, number):
            return False

    return True

# Testa as 'testemunhas' de multiplicidade
# Testemunhas podem determinar se o numero é composto ou dizer nada
def check_witness(a, s, d, number):
    x = pow(a, d, number)
    if x == 1:
        return True
    for i in range(s - 1):
        if x == number - 1:
            return True
        x = pow(x, 2, number)
    return x == number - 1

In [5]:
# Cadeia de testes de primalidade, por ordem de velocidade execução
def is_prime(number):
    if number in prime_cache:
        return True
    for p in prime_cache:
        if ((number % p) == 0):
            return False
    if not is_prime_fermat(number):
        return False

    return is_prime_millerrabin(number, 5)

In [19]:
# Vamos então usar as funções que definimos acima para criar nossos números primos p e q, tais que p != q
def gen_prime(n, window):
    r = 0
    while True:
        r = random.randint(2**(n - window), 2**(n + window))
        if ((r % 2) == 0):
            r -= 1
        if (is_prime(r)):
            return r

def gen_pq(size, window):
    a = gen_prime(size, window)
    while True:
        b = gen_prime(size, window)
        if b != a: break
    return a, b

p, q = gen_pq(16, 1)
print_primes(p, q)


# Números primos gerados
| variável | número primo |
|---|-----|
| p | 83891 |
| q | 56543 |
    

In [8]:
# Escolhidos os p e q, vamos calcular nossas chaves:
# Chave Pública      : e
# Chave Compartilhada: n
# Chave Privada      : d

# Para isso, vamos encontrar primeiro a chave compartilhada n e o totiente phi
n = p * q
phi = (p - 1) * (q - 1)
    
# Com isto, vamos procurar uma chave e tal que phi e a chave pública e são primos entre sí (MDC(e, phi) = 1)
while True:
    e = random.randrange(1, phi)
    if (math.gcd(e, phi) == 1):
        break

In [9]:
# https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm
# Agora, apartir da chave pública e podemos encontrar chave privada d tal que e*d = 1 (mod phi)
# Para isto, vamos usar uma versão do algoritmo extendido de euclides especializada para o cálculo
# do inverso multiplicativo t tal que at = 1 (mod n) com |t| < n
def mult_inv(a, b):
    tab = []
    
    # Seta os valores de t e do resto
    t, nt = 0, 1
    r, nr = b, a
    
    # Enquanto o novo resto for diferente de zero
    while nr != 0:
        tab += [[r, nr, r // nr, r, r - (r//nr) * nr, t - (r//nr) * nt]]
        
        # O novo dividendo é o divisor anterior, o novo divisor é o resto anterior
        quot = r // nr
        
        # Atualiza o t e o resto
        t, nt = nt, t - quot * nt
        r, nr = nr, r - quot * nr
        
    # Queremos resultados positivos, como queremos a relação at = 1 (mod n), e para valores de t negativos t+n < n, então
    # podemos fazer a(t+n) = 1 (mod n) para torná-lo positivo, já que at + an = at (mod n)
    if t < 0: return t+b, tab
    return t, tab


d, tab = mult_inv(e, phi)
print_euctab(tab)
print_keys(e, n, d)


# Passos do Algoritmo Extendido de Euclides
---------------------------------------------------------------------
| Dividendo | Divisor | Quociente | Resto Anterior | Novo Resto | t |
|:---------:|:-------:|:---------:|:--------------:|:----------:|:-:|
    |18017813700|13411540519|1|18017813700|4606273181|-1|
|13411540519|4606273181|2|13411540519|4198994157|3|
|4606273181|4198994157|1|4606273181|407279024|-4|
|4198994157|407279024|10|4198994157|126203917|43|
|407279024|126203917|3|407279024|28667273|-133|
|126203917|28667273|4|126203917|11534825|575|
|28667273|11534825|2|28667273|5597623|-1283|
|11534825|5597623|2|11534825|339579|3141|
|5597623|339579|16|5597623|164359|-51539|
|339579|164359|2|339579|10861|106219|
|164359|10861|15|164359|1444|-1644824|
|10861|1444|7|10861|753|11619987|
|1444|753|1|1444|691|-13264811|
|753|691|1|753|62|24884798|
|691|62|11|691|9|-286997589|
|62|9|6|62|8|1746870332|
|9|8|1|9|1|-2033867921|
|8|1|8|8|0|18017813700|

-----------------------------------------------------------------
    


# Chaves geradas
--------------------------------
| Tipo de Chave | Chave Gerada |
|---------------|--------------|
| Pública       | 13411540519          |
| Compartilhada | 18018090593          |
| Privada       | 15983945779          |
--------------------------------
    

In [10]:
# Com as chaves acima geradas, é então possível encriptar e decriptar mensagens.
message = "Olá Mundo Oculto"

# c = m^e % n
# m = c^d % n
enc = [pow(ord(m), e, n) for m in message]
dec = [chr(pow(c, d, n)) for c in enc]

print_sec(enc, dec)


# Resultados da Encriptação e Decriptação
---------------------------
| Encriptado | Decriptado |
|------------|------------|
    | 8934436674 | O |
| 16756075629 | l |
| 15246335717 | á |
| 6937354227 |   |
| 7433955922 | M |
| 15598410954 | u |
| 1409874110 | n |
| 2950756969 | d |
| 16917994052 | o |
| 6937354227 |   |
| 8934436674 | O |
| 4030518955 | c |
| 15598410954 | u |
| 16756075629 | l |
| 12716095670 | t |
| 16917994052 | o |

---------------------------    
    

In [11]:
# https://www.ibm.com/support/knowledgecenter/en/SSLTBW_2.1.0/com.ibm.zos.v2r1.csfb400/pkcspad.htm
# Então, vamos usar um metodo de padding PKCS#7 definido pelo RFC 5652, isto envolve adicionar um padding para que todos os
# blocos tenham um tamanho bem definido múltiplo de N
# Repare também que o padrão define que padding -sempre- será adicionado a mensagem, mesmo que ela já seja um múltiplo de N
def padding_pkcs7(N, length):
    pad_sz = N - length % N
    return [pad_sz for i in range(pad_sz)]

for i in range(12, 17):
    print('len(msg):', i, '=> padding_pkcs7:', padding_pkcs7(16, i))

len(msg): 12 => padding_pkcs7: [4, 4, 4, 4]
len(msg): 13 => padding_pkcs7: [3, 3, 3]
len(msg): 14 => padding_pkcs7: [2, 2]
len(msg): 15 => padding_pkcs7: [1]
len(msg): 16 => padding_pkcs7: [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]


In [12]:
# Portanto, vamos definir novas maneiras de encriptar/decriptar as mensagens:
def encrypt(msg, pk, n):
    # Os blocos a serem encriptados são a mensagem original + padding
    blocks = [ord(char) for char in msg] + padding(16, len(msg))